simple example. it doesn't pass the whole paper and context to optimizer

In [ ]:
import sys

sys.path.append("C:/Users/vansari/Documents/PhotonicAI")
import pandas as pd
import textgrad as tg
from dotenv import load_dotenv

load_dotenv(dotenv_path="../../.env")

tg.set_backward_engine("gpt-4o", override=True)
# tg.set_backward_engine("gpt-4o-mini", override=True)
# tg.set_backward_engine(tg.get_engine("gpt-3.5-turbo"))

# Step 1: Get an initial response from an LLM.
df = pd.read_parquet("../papers/db/AMF_papers.parquet")
idx = 223
article = df.loc[idx]["text_full"]
print("--> filename: ", df.loc[idx]["filename"])
# 244, 223,

question_string = f"""Is this a single academic article, and not a dissertation or collection of papers (single_article)?
Is the main topic of this article about integrated photonic circuits (topic_photonic)?
If yes, find the photonic components that are used on the chip.
Return a concise list of these photonic components, if any (components_list).
For each component, try to extract: brief spec,
and the number of optical input (N) and output (M) ports denoted by NxM, e.g. 1x2.
Finally, is there an enough information to understand and desrcibe how the on-chip components
are interconnected to form the photonic circuit (circuit_complete)?
Answer in YAML following the template:
single_article: True/False
topic_photonic: True/False
components_list:
  - a 1x1 modulator with MHz speed
  - a 1x2 component ...
  ...
circuit_complete: True/False


INPUT ARTICLE:
{article}
"""

question = tg.Variable(
    question_string, role_description="question to the LLM", requires_grad=False
)

model = tg.BlackboxLLM("gpt-4o")
answer = model(question)

print(30 * "=" + "INITIAL ANSWER")
print(answer)
print(40 * "=")
print(40 * "=")

photonic_critic_prompt = """You are a smart language model expert in photonic integrated circuits.
This YAML file should be an accurate summary of the photonic components presented in the input article.
Evaluate components_list in YAML based on:
- it should only represent photonic components on the chip, and not off the chip.
- does it accurately represent the photonic components?
- is any photonic component missing?
- this should be only a list of photonic components.
- does YAML follow the provided template?
Also evaluate circuit_complete boolean:
- is it correct? can you understand and describe the connection between items in components_list?
You do not propose a new YAML file, only evaluate the existing YAML file critically and give very concise feedback."""
loss_system_prompt = tg.Variable(
    photonic_critic_prompt,
    requires_grad=False,
    role_description="system prompt to the loss function",
)


# x = tg.Variable(str(answer), role_description='The yaml file to optimize', requires_grad=True)
answer.set_role_description("The yaml file to optimize")
answer.requires_grad = True

# system_prompt = tg.Variable("Evaluate the correctness of this sentence", role_description="The system prompt")
loss_fn = tg.TextLoss(loss_system_prompt)


loss = loss_fn(answer)
print(30 * "=" + "LOSS")
print(loss)
loss.backward()
optimizer = tg.TGD(parameters=[answer], verbose=False)
optimizer.step()
print(30 * "=" + "UPDATED ANSWER")
print(answer.value)
print(30 * "=")
answer

In [ ]:
loss.generate_graph()

In [ ]:
# run optimizer again...
optimizer.zero_grad()
loss = loss_fn(answer)
loss.backward()
optimizer = tg.TGD(parameters=[answer], verbose=False)
optimizer.step()
answer